# Implementation of Classification using Deep Learning 
The Machine Learning version of Classification is **2_Classification_Model_Mac.ipynb**

## Following steps were observed:

1. Import the dataframe that was pickled in **2_Classification_Model_Mac.ipynb** (this df contains reviews that have undergone language translation)

2. Built a DL model on this DataFrame and check whether its accuracy is better than the ML model (that was implemented in **2_Classification_Model_Mac.ipynb**


## Task 1

Read **`mcdonalds.csv`** into a pandas DataFrame and do EDA

In [28]:
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import unidecode
import re

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [12]:
# Read the pickled dataframe (see instrauctions above for context)
with open('pickle_and_trained_models/mac.pickle', 'rb') as handle:
    mac = pickle.load(handle)

In [15]:
mac.head(3)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,policies_violated,policies_violated:confidence,city,policies_violated_gold,review,...,BadFood,ScaryMcDs,Cost,Filthy,MissingFood,OrderProblem,SlowService,na,lang,review_city
0,679455653,False,finalized,3,2/21/15 0:36,RudeService\r\nOrderProblem\r\nFilthy,1.0\r\n0.6667\r\n0.6667,Atlanta,NaN,"I'm not a huge mcds lover, but I've been to be...",...,0,0,0,1,0,1,0,0,en,"I'm not a huge mcds lover, but I've been to be..."
1,679455654,False,finalized,3,2/21/15 0:27,RudeService,1,Atlanta,NaN,Terrible customer service. I came in at 9:30pm...,...,0,0,0,0,0,0,0,0,en,Terrible customer service. I came in at 9:30pm...
2,679455655,False,finalized,3,2/21/15 0:26,SlowService\r\nOrderProblem,1.0\r\n1.0,Atlanta,NaN,"First they ""lost"" my order, actually they gave...",...,0,0,0,0,0,1,1,0,en,"First they ""lost"" my order, actually they gave..."


In [16]:
mac.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'policies_violated',
       'policies_violated:confidence', 'city', 'policies_violated_gold',
       'review', 'rude', 'BadFood', 'ScaryMcDs', 'Cost', 'Filthy',
       'MissingFood', 'OrderProblem', 'SlowService', 'na', 'lang',
       'review_city'],
      dtype='object')

In [17]:
# examine the text of the first review
mac.loc[0, 'review']

"I'm not a huge mcds lover, but I've been to better ones. This is by far the worst one I've ever been too! It's filthy inside and if you get drive through they completely screw up your order every time! The staff is terribly unfriendly and nobody seems to care."

## Task 2

Build the DL model for classification

In [21]:
# function to preprocess the text
def cleaner(text):
    newString = text.lower()
    unaccented_string = unidecode.unidecode(newString)
    newString = re.sub("'",'', unaccented_string) 
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = newString.split()
    return (" ".join(tokens)).strip()

# preprocess english text
cleaned_eng = []
for t in mac.review:
    cleaned_eng.append(cleaner(t)) 

mac['review']=cleaned_eng


In [22]:
#define X and y
x_train_text = mac.review
y_train = mac.rude


In [23]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(x_train_text)
vocab_size = len(t.word_index) + 1

# integer encode the documents
x_train_tokens = t.texts_to_sequences(x_train_text )

In [24]:
t.word_index

{'the': 1,
 'i': 2,
 'and': 3,
 'to': 4,
 'a': 5,
 'aa': 6,
 'of': 7,
 'is': 8,
 'this': 9,
 'in': 10,
 'was': 11,
 'it': 12,
 'for': 13,
 'my': 14,
 'they': 15,
 'mcdonalds': 16,
 'that': 17,
 'you': 18,
 'at': 19,
 'not': 20,
 'but': 21,
 'have': 22,
 'on': 23,
 'food': 24,
 'me': 25,
 'order': 26,
 'with': 27,
 'so': 28,
 'one': 29,
 'there': 30,
 'are': 31,
 'drive': 32,
 'get': 33,
 'its': 34,
 'be': 35,
 'just': 36,
 'no': 37,
 'here': 38,
 'up': 39,
 'had': 40,
 'when': 41,
 'service': 42,
 'if': 43,
 'time': 44,
 'go': 45,
 'or': 46,
 'we': 47,
 'as': 48,
 'like': 49,
 'out': 50,
 'thru': 51,
 'place': 52,
 'were': 53,
 'what': 54,
 'all': 55,
 'only': 56,
 'dont': 57,
 'your': 58,
 'because': 59,
 'location': 60,
 'their': 61,
 'an': 62,
 'about': 63,
 'been': 64,
 'from': 65,
 'im': 66,
 'people': 67,
 'do': 68,
 'she': 69,
 'back': 70,
 'even': 71,
 'can': 72,
 'would': 73,
 'always': 74,
 'fries': 75,
 'got': 76,
 'then': 77,
 'through': 78,
 'ive': 79,
 'good': 80,
 'by': 

In [25]:
x_train_text[1]

'terrible customer service i came in at pm and stood in front of the register and no one bothered to say anything or help me for minutes aa there was no one else waiting for their food inside either just outside at the window aa i left and went to chickfila next door and was greeted before i was all the way inside this mcdonalds is also dirty the floor was covered with dropped food obviously filled with surly and unhappy workers'

In [26]:
np.array(x_train_tokens[1])

array([ 321,  117,   42,    2,  185,   10,   19,  376,    3,  892,   10,
        220,    7,    1,  330,    3,   37,   29, 1128,    4,  133,  291,
         46,  435,   25,   13,   89,    6,   30,   11,   37,   29,  260,
        146,   13,   61,   24,  131,  267,   36,  331,   19,    1,   91,
          6,    2,  221,    3,   97,    4, 4027,  213,  361,    3,   11,
       1194,  115,    2,   11,   55,    1,  116,  131,    9,   16,    8,
        144,  275,    1,  465,   11, 1495,   27, 1195,   24,  677,  718,
         27, 2046,    3, 1654,  302])

In [30]:
#read mcdonalds_new.csv into new_comments
path = 'data/mcdonalds_new.csv'

new_comments = pd.read_csv(path)
x_test_text = new_comments.review

# Convert test reviews into integer tokens
x_test_tokens = t.texts_to_sequences(x_test_text)

In [31]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

274

In [32]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9549731182795699

In [33]:
pad = 'pre'
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [34]:
len(x_test_pad[0])

274

In [35]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad)
print(x_train_pad)

# load the whole embedding into memory
embeddings_index = dict()

[[   0    0    0 ...  290    4  323]
 [   0    0    0 ...    3 1654  302]
 [   0    0    0 ...   27    1 2397]
 ...
 [   0    0    0 ... 7835   31  217]
 [2906 4019  101 ...  124   27   12]
 [   0    0    0 ...    5  319  223]]


The below cell loads **pre-trained word embedding (glove)** and created a weight matric for the words in our training docs. We are using it intead of training our own word embedding. This is mainly due to the fact that we have less training data, **so transfer learning via pre-trained word embedding was preferred**

In [39]:
f = open('data/glove.6B.100d.txt', mode='rt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [40]:
idx = t.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [41]:
x_train_text[2]

'first they lost my order actually they gave it to someone one else than took minutes to figure out why i was still waiting for my order they after i was asked what i needed i replied my order they asked for my ticket and the asst mgr looked at the ticket then incompletely filled it i had to ask her to check to see if she filled it correctly she acted as if she couldnt be bothered with that so i asked her again she begrudgingly checked to she did in fact miss something on the ticket so after minutes i finally had my breakfast biscuit platter as i left an woman approached and identified herself as the manager she was dressed as if she had just awoken in an old t shirt and sweat pants she said she had heard what happened and said shed take care of it well why didnt she intervene when she saw i was growing annoyed with the incompetence'

In [42]:
tokens_to_string(x_train_tokens[2])

'first they lost my order actually they gave it to someone one else than took minutes to figure out why i was still waiting for my order they after i was asked what i needed i replied my order they asked for my ticket and the asst mgr looked at the ticket then incompletely filled it i had to ask her to check to see if she filled it correctly she acted as if she couldnt be bothered with that so i asked her again she begrudgingly checked to she did in fact miss something on the ticket so after minutes i finally had my breakfast biscuit platter as i left an woman approached and identified herself as the manager she was dressed as if she had just awoken in an old t shirt and sweat pants she said she had heard what happened and said shed take care of it well why didnt she intervene when she saw i was growing annoyed with the incompetence'

In [43]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_tokens, trainable=False,name='layer_embedding')
model.add(e)
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

W0918 18:49:56.599268 71080 deprecation.py:506] From C:\Users\seqpr\Anaconda3\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0918 18:49:56.675280 71080 deprecation.py:506] From C:\Users\seqpr\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [44]:
optimizer = Adam(lr=1e-3)

In [45]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

W0918 18:50:00.175090 71080 deprecation.py:323] From C:\Users\seqpr\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 274, 100)          787700    
_________________________________________________________________
gru (GRU)                    (None, 274, 16)           5616      
_________________________________________________________________
gru_1 (GRU)                  (None, 274, 8)            600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 794,077
Trainable params: 6,377
Non-trainable params: 787,700
_________________________________________________________________


In [47]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=3)

In [48]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=50, batch_size=64,callbacks = [early_stopping_monitor])

Train on 1400 samples, validate on 74 samples
Epoch 1/50
1400/1400 [==============================] - 12s 9ms/sample - loss: 0.6566 - acc: 0.6450 - val_loss: 0.6767 - val_acc: 0.6216
Epoch 2/50
1400/1400 [==============================] - 7s 5ms/sample - loss: 0.6443 - acc: 0.6564 - val_loss: 0.6649 - val_acc: 0.6216
Epoch 3/50
1400/1400 [==============================] - 9s 7ms/sample - loss: 0.6402 - acc: 0.6564 - val_loss: 0.6635 - val_acc: 0.6216
Epoch 4/50
1400/1400 [==============================] - 7s 5ms/sample - loss: 0.6337 - acc: 0.6564 - val_loss: 0.6585 - val_acc: 0.6216
Epoch 5/50
1400/1400 [==============================] - 8s 6ms/sample - loss: 0.6237 - acc: 0.6586 - val_loss: 0.6422 - val_acc: 0.6216
Epoch 6/50
1400/1400 [==============================] - 9s 6ms/sample - loss: 0.6010 - acc: 0.6736 - val_loss: 0.6187 - val_acc: 0.6486
Epoch 7/50
1400/1400 [==============================] - 10s 7ms/sample - loss: 0.5689 - acc: 0.7157 - val_loss: 0.5841 - val_acc: 0.6351


In [49]:
%%time
result = model.predict_proba(x_test_pad)

Wall time: 443 ms


In [51]:
pd.set_option('display.max_colwidth', 1000)
df_severity = pd.DataFrame({'comment':x_test_text, 'severity':result.ravel()}).sort_values('severity', ascending=False)
df_severity

,comment,severity
11,worst experience of a lifetime. Dishonest staff! Get ready for ridiculous service. Better to go to other places,0.891628
2,Ghetto lady helped me at the drive thru. Very rude and disrespectful to the co workers. Never coming back. Yuck!,0.875200
7,My friend and I stopped in to get a late night snack and we were refused service. The store claimed to be 24 hours and the manager was standing right there doing paper work but would not help us. The cashier was only concerned with doing things for the drive thru and said that the manager said he wasn't allowed to help us. We thought it was a joke at first but when realized it wasn't we said goodbye and they just let us leave. I work in a restaurant and this is by far the worst service I have ever seen. I know it was late and maybe they didn't want to be there but it was completely ridiculous. I think the manager should be fired.,0.868014
10,"disgusting staff, never going to here again!",0.862801
0,"Went through the drive through and ordered a #10 (cripsy sweet chili chicken wrap) without fries- the lady couldn't understand that I did not want fries and charged me for them anyways. I got the wrong order- a chicken sandwich and a large fries- my boyfriend took it back inside to get the correct order. The gentleman that ordered the chicken sandwich was standing there as well and she took the bag from my bf- glanced at the insides and handed it to the man without even offering to replace. I mean with all the scares about viruses going around... ugh DISGUSTING SERVICE. Then when she gave him the correct order my wrap not only had the sweet chili sauce on it, but the nasty (just not my first choice) ranch dressing on it!!!! I mean seriously... how lazy can you get!!!! I worked at McDonalds in Texas when I was 17 for about 8 months and I guess I was spoiled with good management. This was absolutely ridiculous. I was beyond disappointed.",0.846994
1,"Phenomenal experience. Efficient and friendly staff. Clean restrooms, good, fast service and bilingual staff. One of the best restaurants in the chain.",0.605020
6,"This specific McDonald's is the bar I hold all other fast food joints to now. Been working in this area for 3 years now and gone to this location many times for drive-through pickup. Service is always fast, food comes out right, and the staff is extremely warm and polite.",0.536876
12,"Yucky, to say the least. I'd be fired if I was as disrespectful in my workplace",0.442984
13,Offensive service. This is definitely mast last time at this unworthy place.,0.409282
8,"Friendly people but completely unable to deliver what was ordered at the drive through. Out of my last 6 orders they got it right 3 times. Incidentally, the billing was always correct - they just could not read the order and deliver. Very frustrating!",0.232197


# **Comments:**

- The model correctly ranked all **"rude service" comments** (11, 2, 7 and 7) above all other comments
- The  comments (1 and 6) which were **negative but not about rude service** were more 0.5, which is not correct
- Other comments which were **negative but not about rude service** were ranked less than 0.5, which is correct

**Conclusion:** DL needs more data to train, and we have just about 1500 records (and the language had a many slang words), and hence better results could not be achieved despite using pre-trained model (transfer learning)

# **PLEASE NOTE:**

- Accuracy of 81% was got on validation set, and the model was tested on unseen sample with mediocre results due to **less training data**
- train-test split was **not done** since model performed badly with less train-”test” data. Less data was the limiting fact. 
